&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](Start_Here.ipynb)
    
    
[Previous Notebook](Introduction_to_Multi-DNN_pipeline.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&ensp;
[1](Introduction_to_Deepstream_and_Gstreamer.ipynb)
[2](Getting_started_with_Deepstream_Pipeline.ipynb)
[3](Introduction_to_Multi-DNN_pipeline.ipynb)
[4]
[5](Multi-stream_Multi_DNN.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Multi-stream_Multi_DNN.ipynb)


# Multi-stream pipeline

In this notebook, we will learn to build a pipeline that can take in two stream and batch them to run inference and process them together. 

**Contents of this Notebook :**

- [Nvmultistreamtiler](#Nvmultistreamtiler) 
- [Building the pipeline](#Building-the-pipeline)
  
  
![test3](images/test3.png)

We will be using `nvmultistreamtiler` for this application to composite a 2D tile from batched buffers.

### Nvmultistreamtiler

The Gst-nvmultistreamtiler plugin composites a 2D tile from batched buffers. The plugin accepts batched NV12/RGBA data from upstream components. The plugin composites the tile based on stream IDs, obtained from NvDsBatchMeta and NvDsFrameMeta in row-major order (starting from source 0, left to right across the top row, then across the next row). Each source frame is scaled to the corresponding location in the tiled output buffer. The plugin can reconfigure if a new source is added and it exceeds the space allocated for tiles. It also maintains a cache of old frames to avoid display flicker if one source has a lower frame rate than other sources.

![test3](images/nvmultistreamtiler.png)


## Building the pipeline

We will the using batched on the 4-class DNN network. Rest of the notebook remains mostly similar to primary notebook.  

![test1](images/test1.png)

In [1]:
# Import required libraries 
import sys
sys.path.append('../source_code')
import gi
import configparser
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst
from gi.repository import GLib
from ctypes import *
import time
import sys
import math
import platform
from common.bus_call import bus_call
from common.FPS import GETFPS
import pyds


# Define variables to be used later
fps_streams={}

PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

MUXER_OUTPUT_WIDTH=1920
MUXER_OUTPUT_HEIGHT=1080

TILED_OUTPUT_WIDTH=1920
TILED_OUTPUT_HEIGHT=1080
OSD_PROCESS_MODE= 0
OSD_DISPLAY_TEXT= 0
pgie_classes_str= ["Vehicle", "TwoWheeler", "Person","RoadSign"]

# Define Input and output Stream information 
num_sources = 2 
INPUT_VIDEO_1 = '/opt/nvidia/deepstream/deepstream-6.0/samples/streams/sample_720p.h264'
INPUT_VIDEO_2 = '/opt/nvidia/deepstream/deepstream-6.0/samples/streams/sample_720p.h264'
OUTPUT_VIDEO_NAME = "../source_code/N3/ds_out.mp4"

We define a function `make_elm_or_print_err()` to create our elements and report any errors if the creation fails.

Elements are created using the `Gst.ElementFactory.make()` function as part of Gstreamer library.

In [2]:
## Make Element or Print Error and any other detail
def make_elm_or_print_err(factoryname, name, printedname, detail=""):
  print("Creating", printedname)
  elm = Gst.ElementFactory.make(factoryname, name)
  if not elm:
     sys.stderr.write("Unable to create " + printedname + " \n")
  if detail:
     sys.stderr.write(detail)
  return elm

#### Initialise GStreamer and Create an Empty Pipeline

In [3]:
for i in range(0,num_sources):
        fps_streams["stream{0}".format(i)]=GETFPS(i)

# Standard GStreamer initialization
GObject.threads_init()
Gst.init(None)

# Create gstreamer elements */
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n ")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")


Creating Pipeline 
 


/tmp/ipykernel_9398/4208069694.py:5: PyGIDeprecationWarning: Since version 3.11, calling threads_init is no longer needed. See: https://wiki.gnome.org/PyGObject/Threading
  GObject.threads_init()


#### Create Elements that are required for our pipeline

Compared to the first notebook , we use a lot of queues in this notebook to buffer data when it moves from one plugin to another.

In [4]:
########### Create Elements required for the Pipeline ########### 

######### Defining Stream 1 
# Source element for reading from the file
source1 = make_elm_or_print_err("filesrc", "file-source-1",'file-source-1')
# Since the data format in the input file is elementary h264 stream,we need a h264parser
h264parser1 = make_elm_or_print_err("h264parse", "h264-parser-1","h264-parser-1")
# Use nvdec_h264 for hardware accelerated decode on GPU
decoder1 = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder-1","nvv4l2-decoder-1")
   
##########

########## Defining Stream 2 
# Source element for reading from the file
source2 = make_elm_or_print_err("filesrc", "file-source-2","file-source-2")
# Since the data format in the input file is elementary h264 stream, we need a h264parser
print("Creating H264Parser \n")
h264parser2 = make_elm_or_print_err("h264parse", "h264-parser-2", "h264-parser-2")
# Use nvdec_h264 for hardware accelerated decode on GPU
print("Creating Decoder \n")
decoder2 = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder-2","nvv4l2-decoder-2")
########### 
    
# Create nvstreammux instance to form batches from one or more sources.
streammux = make_elm_or_print_err("nvstreammux", "Stream-muxer","Stream-muxer") 
# Creating primary Inference 
pgie = make_elm_or_print_err("nvinfer", "primary-inference","primary-inference")
# Creating Tiler to present more than one streams
tiler=make_elm_or_print_err("nvmultistreamtiler", "nvtiler","nvtiler")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor","nvvidconv")
# Create OSD to draw on the converted RGBA buffer
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay","nvosd")
# Creating queue's to buffer incoming data from pgie
queue1=make_elm_or_print_err("queue","queue1","queue1")
# Creating queue's to buffer incoming data from tiler
queue2=make_elm_or_print_err("queue","queue2","queue2")
# Creating queue's to buffer incoming data from nvvidconv
queue3=make_elm_or_print_err("queue","queue3","queue3")
# Creating queue's to buffer incoming data from nvosd
queue4=make_elm_or_print_err("queue","queue4","queue4")
# Creating queue's to buffer incoming data from nvvidconv2
queue5=make_elm_or_print_err("queue","queue5","queue5")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2","nvvidconv2")
# Place an encoder instead of OSD to save as video file
encoder = make_elm_or_print_err("avenc_mpeg4", "encoder", "Encoder")
# Parse output from Encoder 
codeparser = make_elm_or_print_err("mpeg4videoparse", "mpeg4-parser", 'Code Parser')
# Create a container
container = make_elm_or_print_err("qtmux", "qtmux", "Container")
# Create Sink for storing the output 
sink = make_elm_or_print_err("filesink", "filesink", "Sink")

Creating file-source-1
Creating h264-parser-1
Creating nvv4l2-decoder-1
Creating file-source-2
Creating H264Parser 

Creating h264-parser-2
Creating Decoder 

Creating nvv4l2-decoder-2
Creating Stream-muxer
Creating primary-inference
Creating nvtiler
Creating nvvidconv
Creating nvosd
Creating queue1
Creating queue2
Creating queue3
Creating queue4
Creating queue5
Creating nvvidconv2
Creating Encoder
Creating Code Parser
Creating Container
Creating Sink


Now that we have created the elements ,we can now set various properties for out pipeline at this point. 

Configuration file : [pgie](../source_code/N3/dstest3_pgie_config.txt)

In [6]:
############ Set properties for the Elements ############
# Set Input Video files 
source1.set_property('location', INPUT_VIDEO_1)
source2.set_property('location', INPUT_VIDEO_2)
# Set Input Width , Height and Batch Size 
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', num_sources)
# Timeout in microseconds to wait after the first buffer is available 
# to push the batch even if a complete batch is not formed.
streammux.set_property('batched-push-timeout', 4000000)
# Set configuration file for nvinfer 
pgie.set_property('config-file-path', "../source_code/N3/dstest3_pgie_config.txt")
pgie_batch_size=pgie.get_property("batch-size")
print("PGIE batch size :",end='')
print(pgie_batch_size)
if(pgie_batch_size != num_sources):
    print("WARNING: Overriding infer-config batch-size",pgie_batch_size," with number of sources ", num_sources," \n")
    pgie.set_property("batch-size",num_sources)
    
# Set display configurations for nvmultistreamtiler    
tiler_rows=int(math.sqrt(num_sources))
tiler_columns=int(math.ceil((1.0*num_sources)/tiler_rows))
tiler.set_property("rows",tiler_rows)
tiler.set_property("columns",tiler_columns)
tiler.set_property("width", TILED_OUTPUT_WIDTH)
tiler.set_property("height", TILED_OUTPUT_HEIGHT)

# Set encoding properties and Sink configs
encoder.set_property("bitrate", 2000000)
sink.set_property("location", OUTPUT_VIDEO_NAME)
sink.set_property("sync", 0)
sink.set_property("async", 0)


PGIE batch size :4



We now link all the elements in the order we prefer and create Gstreamer bus to feed all messages through it. 

In [7]:
########## Add and Link ELements in the Pipeline ########## 

print("Adding elements to Pipeline \n")
pipeline.add(source1)
pipeline.add(h264parser1)
pipeline.add(decoder1)
pipeline.add(source2)
pipeline.add(h264parser2)
pipeline.add(decoder2)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(tiler)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(queue1)
pipeline.add(queue2)
pipeline.add(queue3)
pipeline.add(queue4)
pipeline.add(queue5)
pipeline.add(nvvidconv2)
pipeline.add(encoder)
pipeline.add(codeparser)
pipeline.add(container)
pipeline.add(sink)

print("Linking elements in the Pipeline \n")

source1.link(h264parser1)
h264parser1.link(decoder1)


###### Create Sink pad and connect to decoder's source pad 
sinkpad1 = streammux.get_request_pad("sink_0")
if not sinkpad1:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
    
srcpad1 = decoder1.get_static_pad("src")
if not srcpad1:
    sys.stderr.write(" Unable to get source pad of decoder \n")
    
srcpad1.link(sinkpad1)

###### Create Sink pad and connect to decoder's source pad 
source2.link(h264parser2)
h264parser2.link(decoder2)

sinkpad2 = streammux.get_request_pad("sink_1")
if not sinkpad2:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
    
srcpad2 = decoder2.get_static_pad("src")
if not srcpad2:
    sys.stderr.write(" Unable to get source pad of decoder \n")
    
srcpad2.link(sinkpad2)



streammux.link(queue1)
queue1.link(pgie)
pgie.link(queue2)
queue2.link(tiler)
tiler.link(queue3)
queue3.link(nvvidconv)
nvvidconv.link(queue4)
queue4.link(nvosd)
nvosd.link(queue5)
queue5.link(nvvidconv2)
nvvidconv2.link(encoder)
encoder.link(codeparser)
codeparser.link(container)
container.link(sink)


Adding elements to Pipeline 

Linking elements in the Pipeline 



True

In [8]:
# create an event loop and feed gstreamer bus mesages to it
loop = GObject.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)


/tmp/ipykernel_9398/2016700122.py:2: PyGIDeprecationWarning: GObject.MainLoop is deprecated; use GLib.MainLoop instead
  loop = GObject.MainLoop()


1

Our pipeline now carries the metadata forward but we have not done anything with it until now, but as mentoioned in the above pipeline diagram , we will now create a callback function to write relevant data on the frame once called and create a src pad in the pgie element to call the function. 

This callback function is the same as used in the previous notebook but **notice the location that we have attached the callback function to.**  We make this change because after every frame goes through the `pgie` , it carries the metadata generated from the primary inference , but if you look at the image above on `nvmultistreamtiler` elemenet, we can notice that the output is a *transformed metadata*, hence the metadata changes when it goes through the `nvmultistreamtiler` element because of which we have the callback function attached to the source pad of `pgie` element and not to the nvosd element like in the previous notebooks.

For a better understanding on how exactly `nvmultistreamtiler` changes the metadata, try attaching the callback function to the source pad of tiler element like mentioned below, the same details can also be found in it's [documentation](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/index.html#page/DeepStream%20Plugins%20Development%20Guide/deepstream_plugin_details.html#wwpID0E0PX0HA).


```bash
# Change line from 
tiler_src_pad=pgie.get_static_pad("src")
# to 
tiler_src_pad=tiler.get_static_pad("src")
```

In [9]:
# tiler_sink_pad_buffer_probe  will extract metadata received on OSD sink pad
# and update params for drawing rectangle, object information etc.
def tiler_src_pad_buffer_probe(pad,info,u_data):
    #Intiallizing object counter with 0.
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE:0,
        PGIE_CLASS_ID_PERSON:0,
        PGIE_CLASS_ID_BICYCLE:0,
        PGIE_CLASS_ID_ROADSIGN:0
    }
    # Set frame_number & rectangles to draw as 0 
    frame_number=0
    num_rects=0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        # Get frame number , number of rectables to draw and object metadata
        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            # Increment Object class by 1 and Set Box border to Red color     
            obj_counter[obj_meta.class_id] += 1
            obj_meta.rect_params.border_color.set(0.0, 0.0, 1.0, 0.0)
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        ################## Setting Metadata Display configruation ############### 
        # Acquiring a display meta object.
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Vehicle_count={} Person_count={}".format(frame_number, num_rects, obj_counter[PGIE_CLASS_ID_VEHICLE], obj_counter[PGIE_CLASS_ID_PERSON])
        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        # Font , font-color and font-size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        # Set(red, green, blue, alpha); Set to White
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
        # Text background color
        py_nvosd_text_params.set_bg_clr = 1
        # Set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Using pyds.get_string() to get display_text as string to print in notebook
        print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        ############################################################################
        # Get frame rate through this probe
        fps_streams["stream{0}".format(frame_meta.pad_index)].get_fps()
        try:
            l_frame=l_frame.next
        except StopIteration:
            break

    return Gst.PadProbeReturn.OK


In [10]:
tiler_src_pad=pgie.get_static_pad("src")
if not tiler_src_pad:
    sys.stderr.write(" Unable to get src pad \n")
else:
    tiler_src_pad.add_probe(Gst.PadProbeType.BUFFER, tiler_src_pad_buffer_probe, 0)

Now with everything defined , we can start the playback and listen the events.

In [11]:
# List the sources
print("Now playing...")
start_time = time.time()
print("Starting pipeline \n")
# start play back and listed to events		
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
# cleanup
print("Exiting app\n")
pipeline.set_state(Gst.State.NULL)
print("--- %s seconds ---" % (time.time() - start_time))

Now playing...
Starting pipeline 



0:00:00.326187003  9398      0x1f76690 WARN                    v4l2 gstv4l2object.c:3051:gst_v4l2_object_get_nearest_size:<nvv4l2-decoder-1:sink> Unable to try format: Unknown error -1
0:00:00.326213803  9398      0x1f76690 WARN                    v4l2 gstv4l2object.c:2937:gst_v4l2_object_probe_caps_for_format:<nvv4l2-decoder-1:sink> Could not probe minimum capture size for pixelformat MJPG
0:00:00.326217843  9398      0x1f76690 WARN                    v4l2 gstv4l2object.c:3051:gst_v4l2_object_get_nearest_size:<nvv4l2-decoder-1:sink> Unable to try format: Unknown error -1
0:00:00.326221123  9398      0x1f76690 WARN                    v4l2 gstv4l2object.c:2943:gst_v4l2_object_probe_caps_for_format:<nvv4l2-decoder-1:sink> Could not probe maximum capture size for pixelformat MJPG
0:00:00.326238033  9398      0x1f76690 WARN                    v4l2 gstv4l2object.c:3051:gst_v4l2_object_get_nearest_size:<nvv4l2-decoder-1:sink> Unable to try format: Unknown error -1
0:00:00.326241363  9398    

0:00:22.311358201  9398      0x1f76690 INFO                 nvinfer gstnvinfer.cpp:638:gst_nvinfer_logger:<primary-inference> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::buildModel() <nvdsinfer_context_impl.cpp:1947> [UID = 1]: serialize cuda engine to file: /opt/nvidia/deepstream/deepstream-6.0/samples/models/Primary_Detector/resnet10.caffemodel_b2_gpu0_int8.engine successfully
0:00:22.320239796  9398      0x1f76690 INFO                 nvinfer gstnvinfer_impl.cpp:313:notifyLoadModelStatus:<primary-inference> [UID 1]: Load new model:../source_code/N3/dstest3_pgie_config.txt sucessfully
0:00:22.321063118  9398      0x1f76690 WARN                 basesrc gstbasesrc.c:3600:gst_base_src_start_complete:<file-source-1> pad not activated yet
0:00:22.321296788  9398      0x1f76690 WARN                 basesrc gstbasesrc.c:3600:gst_base_src_start_complete:<file-source-2> pad not activated yet
0:00:22.434392117  9398      0x33df700 WARN            v4l2videodec gstv4l2videodec.c:168

Frame Number=0 Number of Objects=13 Vehicle_count=9 Person_count=4
Frame Number=0 Number of Objects=13 Vehicle_count=18 Person_count=8
Frame Number=1 Number of Objects=11 Vehicle_count=8 Person_count=3
Frame Number=1 Number of Objects=11 Vehicle_count=16 Person_count=6
Frame Number=2 Number of Objects=11 Vehicle_count=7 Person_count=4
Frame Number=2 Number of Objects=11 Vehicle_count=14 Person_count=8
Frame Number=3 Number of Objects=11 Vehicle_count=7 Person_count=4
Frame Number=3 Number of Objects=11 Vehicle_count=14 Person_count=8
Frame Number=4 Number of Objects=11 Vehicle_count=8 Person_count=3
Frame Number=4 Number of Objects=11 Vehicle_count=16 Person_count=6
Frame Number=5 Number of Objects=12 Vehicle_count=8 Person_count=4
Frame Number=5 Number of Objects=12 Vehicle_count=16 Person_count=8
Frame Number=6 Number of Objects=11 Vehicle_count=7 Person_count=4
Frame Number=6 Number of Objects=11 Vehicle_count=14 Person_count=8
Frame Number=7 Number of Objects=11 Vehicle_count=7 Per

0:00:22.532696330  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:22.553870167  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:22.572458310  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:22.590526361  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:22.605848338  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:22.620749024  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:22.634834849  9398      0x28b5580 WARN                   libav gstavvidenc.c:

...

Frame Number=1398 Number of Objects=19 Vehicle_count=17 Person_count=2
Frame Number=1398 Number of Objects=19 Vehicle_count=34 Person_count=4
Frame Number=1399 Number of Objects=13 Vehicle_count=13 Person_count=0
Frame Number=1399 Number of Objects=13 Vehicle_count=26 Person_count=0
Frame Number=1400 Number of Objects=14 Vehicle_count=14 Person_count=0
Frame Number=1400 Number of Objects=14 Vehicle_count=28 Person_count=0
Frame Number=1401 Number of Objects=16 Vehicle_count=16 Person_count=0
Frame Number=1401 Number of Objects=16 Vehicle_count=32 Person_count=0
Frame Number=1402 Number of Objects=17 Vehicle_count=17 Person_count=0
Frame Number=1402 Number of Objects=17 Vehicle_count=34 Person_count=0
Frame Number=1403 Number of Objects=15 Vehicle_count=15 Person_count=0
Frame Number=1403 Number of Objects=15 Vehicle_count=30 Person_count=0
Frame Number=1404 Number of Objects=14 Vehicle_count=14 Person_count=0
Frame Number=1404 Number of Objects=14 Vehicle_count=28 Person_count=0
Frame 

0:00:37.992632090  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.002432476  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.012322792  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.022009988  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.031889444  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.041854860  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.051924127  9398      0x28b5580 WARN                   libav gstavvidenc.c:

Frame Number=1420 Number of Objects=16 Vehicle_count=12 Person_count=4
Frame Number=1420 Number of Objects=16 Vehicle_count=24 Person_count=8
Frame Number=1421 Number of Objects=10 Vehicle_count=8 Person_count=2
Frame Number=1421 Number of Objects=10 Vehicle_count=16 Person_count=4
Frame Number=1422 Number of Objects=10 Vehicle_count=7 Person_count=3
Frame Number=1422 Number of Objects=10 Vehicle_count=14 Person_count=6
Frame Number=1423 Number of Objects=12 Vehicle_count=10 Person_count=2
Frame Number=1423 Number of Objects=12 Vehicle_count=20 Person_count=4
Frame Number=1424 Number of Objects=14 Vehicle_count=11 Person_count=3
Frame Number=1424 Number of Objects=14 Vehicle_count=22 Person_count=6
Frame Number=1425 Number of Objects=11 Vehicle_count=9 Person_count=2
Frame Number=1425 Number of Objects=11 Vehicle_count=18 Person_count=4
Frame Number=1426 Number of Objects=11 Vehicle_count=9 Person_count=2
Frame Number=1426 Number of Objects=11 Vehicle_count=18 Person_count=4
Frame Numb

0:00:38.204757696  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.212774559  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.224396458  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.235708337  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.246962785  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.257949993  9398      0x28b5580 WARN                   libav gstavvidenc.c:520:stereo_gst_to_av: Unsupported multiview mode - no mapping in libav
0:00:38.268478830  9398      0x28b5580 WARN                   libav gstavvidenc.c:

Frame Number=1439 Number of Objects=14 Vehicle_count=12 Person_count=2
Frame Number=1439 Number of Objects=14 Vehicle_count=24 Person_count=4
Frame Number=1440 Number of Objects=14 Vehicle_count=13 Person_count=1
Frame Number=1440 Number of Objects=14 Vehicle_count=26 Person_count=2
Frame Number=1441 Number of Objects=0 Vehicle_count=0 Person_count=0
Frame Number=1441 Number of Objects=0 Vehicle_count=0 Person_count=0
End-of-stream
Exiting app

--- 38.21422004699707 seconds ---


In [12]:
# Convert video profile to be compatible with Jupyter notebook
!ffmpeg -loglevel panic -y -an -i ../source_code/N3/ds_out.mp4 -vcodec libx264 -pix_fmt yuv420p -profile:v baseline -level 3 ../source_code/N3/output.mp4

In [13]:
# Display the Output
from IPython.display import HTML
HTML("""
 <video width="960" height="540" controls>
 <source src="../source_code/N3/output.mp4"
 </video>
""".format())

## Licensing
  
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).

[Previous Notebook](Introduction_to_Multi-DNN_pipeline.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&ensp;
[1](Introduction_to_Deepstream_and_Gstreamer.ipynb)
[2](Getting_started_with_Deepstream_Pipeline.ipynb)
[3](Introduction_to_Multi-DNN_pipeline.ipynb)
[4]
[5](Multi-stream_Multi_DNN.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Multi-stream_Multi_DNN.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;
[Home Page](Start_Here.ipynb)
    
 